<a href="https://colab.research.google.com/github/kgautam2103/Eventbrite_automation/blob/main/create_recurring_event_eventbrite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime

#define global variables
auth_token = ''
prod_url = 'https://www.eventbriteapi.com/v3/'
prod_proxy_url= 'https://private-anon-3ec5fc757f-eventbriteapiv3public.apiary-proxy.com/v3/'
mock_url='https://private-anon-86d996fd56-eventbriteapiv3public.apiary-mock.com/v3/'
org = 'organizations/'
venues= '/venues/'
events= 'events/'
copy= '/copy/'
publish= '/publish/'
schedule = '/schedules/'
#start_date='2021-08-24T00:45:00Z'
#end_date='2021-08-24T01:45:00Z'
start_date = input('enter the start date as 2021-08-24T00:45:00Z : ')
end_date = input('enter the end date as 2021-08-24T00:45:00Z : ')
duration = input('enter the duration for the event in seconds for 1 hour it should be 3600 : ')
frequency = input('enter DAILY/WEEKLY/MONTHLY : ')
frequency_count = input('enter the recurrence count : ')
inputdata = input('enter the sheet for execution like inputdata_US: ')
#end_date = input('enter the end date as 2021-08-24T00:45:00Z : ')
current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_recurring_event_output'
duration = int(duration)
recurrence_string = "DTSTART:"+start_date+"\nRRULE:FREQ="+frequency+";COUNT="+frequency_count
print(recurrence_string)
counter=1

authorization = 'Bearer '+auth_token
headers = {
        'Authorization': authorization,
        'Content-Type': 'application/json'
    }

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ryMfzfy6u2N0V4qg1B-xzRYTaNcs_e4o-DBddru1raI/edit#gid=0')

# create and Open our new sheet and add some data.
sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['event_url','event_id','is_published','venue_id','city','region','postal_code','country','org_id','venue_error','event_error'],1)


input_sheet = wb.worksheet(inputdata)
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['name',	'capacity','address1','address2','city','region',
                                                'postal_code','country','timezone','org_id','sample_event_id','replace_city'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

for index,row in inputdata_df.iterrows():
  counter = counter+1
  venue_id=""
  venue_error=""
  event_id = ""
  event_error = ""
  copy_error = ""
  update_error = ""
  schedule_error = ""
  publish_error = ""
  is_published = False
  cell_value_list = []
  address_details={
        "address_1": str(row['address1']),
        "address_2": str(row['address2']),
        "city": str(row['city']),
        "region": str(row['region']),
        "postal_code": str(row['postal_code']),
        "country": str(row['country'])
    }

  venue_details = {
        "name": str(row['name']),
        "capacity": str(row['capacity']),
        "address":address_details
        }

  final_venue_body = {
        "venue": venue_details
    }

  organization_id = str(row['org_id']).strip()
  venue_url = prod_url + org + organization_id + venues
  venue_values = json.dumps(final_venue_body)

  venue_response = requests.post(venue_url,data=venue_values,headers=headers)
  venue_status = venue_response.status_code

  if venue_status == 200:
    venue_id = str(venue_response.json()['id'])
  else:
    venue_error = venue_response.text

  print("venue_id "+venue_id)
  print("venue_status "+str(venue_status))
  print("venue_error "+venue_error)

  city_name = str(row['city'])
  sample_event_id = str(row['sample_event_id'])
  retrieve_event_url = prod_url+events+sample_event_id
  retrieve_response = requests.get(retrieve_event_url,headers=headers)

  event_name = str(retrieve_response.json()['name']['text'])
  event_desc = str(retrieve_response.json()['description']['text'])
  new_event_name = event_name.replace(str(row['replace_city']),city_name)
  timezone = str(row['timezone']).strip()
  copy_request_body = {"timezone": timezone}

  copy_event_values = json.dumps(copy_request_body)

  copy_event_url = prod_url+events+sample_event_id+copy
  copy_response = requests.post(copy_event_url,data=copy_event_values, headers=headers)
  copy_status = copy_response.status_code

  if copy_status == 200:
    event_id = str(copy_response.json()['id'])
  else:
    copy_error = copy_response.text

  print("event_id "+event_id)
  print("copy_status "+str(copy_status))
  print("copy_error "+copy_error)

  update_event_url = prod_url+events+event_id+"/"
  create_schedule_url = prod_url+events+event_id+schedule
  publish_event_url = prod_url+events+event_id+publish


  print(new_event_name)

  update_request_body = {"event": {"name": {
      "html": new_event_name},"venue_id": venue_id
    }
    }

  schedule_event_body = {  "schedule": {
    "occurrence_duration": duration,
    "recurrence_rule": recurrence_string
    }
  }

  update_event_values = json.dumps(update_request_body)
  #print(update_event_values)
  create_schedule_event_values = json.dumps(schedule_event_body)
  print(create_schedule_event_values)

  update_response = requests.post(update_event_url, data=update_event_values, headers=headers)
  update_status = update_response.status_code

  print(update_response.json())
  print("update_status "+str(update_status))

  if update_status == 200:
    schedule_event_response = requests.post(create_schedule_url, data=create_schedule_event_values, headers=headers)
    schedule_event_status = schedule_event_response.status_code
    if schedule_event_status == 200:
      publish_response = requests.post(publish_event_url,headers=headers)
      publish_status = publish_response.status_code
      if publish_status == 200:
        is_published=True
      else:
        publish_error = publish_response.text
    else:
      schedule_error = schedule_event_response.text
  else:
    update_error = update_response.text

  print(is_published)
  event_error = copy_error+update_error+schedule_error+publish_error
  print("event_error "+event_error)
  event_url = "https://www.eventbrite.com/myevent?eid="+event_id
  cell_value_list = [event_url,event_id,is_published,venue_id,city_name,str(row['region']),str(row['postal_code']),str(row['country']),organization_id,venue_error,event_error]
  output_sheet.insert_row(cell_value_list,counter)



enter the start date as 2021-08-24T00:45:00Z : 2024-01-28T01:45:00Z
enter the end date as 2021-08-24T00:45:00Z : 2024-01-30T02:45:00Z
enter the duration for the event in seconds for 1 hour it should be 3600 : 3600
enter DAILY/WEEKLY/MONTHLY : DAILY
enter the recurrence count : 3
enter the sheet for execution like inputdata_US: Inputdata_NA_mountain
DTSTART:2024-01-28T01:45:00Z
RRULE:FREQ=DAILY;COUNT=3
venue_id 188077249
venue_status 200
venue_error 
event_id 776167105707
copy_status 200
copy_error 
Phoenix: Free Guided Meditation for 21 days for Freedom from Thoughts!
{"schedule": {"occurrence_duration": 3600, "recurrence_rule": "DTSTART:2024-01-28T01:45:00Z\nRRULE:FREQ=DAILY;COUNT=3"}}
{'name': {'text': 'Phoenix: Free Guided Meditation for 21 days for Freedom from Thoughts!', 'html': 'Phoenix: Free Guided Meditation for 21 days for Freedom from Thoughts!'}, 'description': {'text': 'Tired of thoughts and same old? Why not try a completely free 21 days guided meditation course for peace